#### Imports

In [1]:
import sys
import nltk.sentiment
import pandas as pd
import nltk

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

c:\Users\George Local\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\George Local\.cache\huggingface\hub\models--distilbert--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

#### Read the dataset

In [2]:
data = pd.read_csv("output.csv", keep_default_na=False)

#### Define the text preprocessing functions

In [ ]:
def pre_process(text):

    #First tokenize the text to continue
    tokens = nltk.tokenize.word_tokenize(text.lower())

    #remove stop words:
    stopword_list = nltk.corpus.stopwords.words('english')

    filtered_tokens = [token for token in tokens if token not in stopword_list]

    lemmatizer = nltk.stem.WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]


    processed_text = ' '.join(lemmatized_tokens)

    return processed_text



def sentiment_analysis_nltk(processed_text):

    analyzer = nltk.sentiment.vader.SentimentIntensityAnalyzer()

    score = analyzer.polarity_scores(processed_text)

    return score['compound']

def sentiment_analysis_huggingface(text):
    result = sentiment_pipeline(text)
    return result

#### Run the functions on the dataframe

In [ ]:
data['Processed text'] = data['Content'].apply(pre_process)
data['Sentiment_nltk'] = data['Processed text'].apply(sentiment_analysis_nltk)

data['Sentiment_huggingface'] = data['Content'].apply(sentiment_analysis_huggingface)

#### Save the results in a new csv file

In [ ]:
data.to_csv('test.csv')